## Perform single-cell pycytominer pipelines

Following single-cell curation with cytotable, we create single-cell profiles by applying the following steps:

1. annotation
2. normalization
3. feature_selection

Additionally, we create bulk profiles following feature selection.
We call this "Cameron's Method".

4. Aggregate (to form bulk, after single-cell processing)

In [1]:
import pathlib
import yaml
import pprint

import pandas as pd

from pycytominer import aggregate, annotate, normalize, feature_select
from pycytominer.cyto_utils import output, infer_cp_features

In [2]:
# Set the data level to process (converted/raw or cleaned/QC)
data_level = "cleaned"

# Set feature selection operations
feature_select_ops = [
    "variance_threshold",
    "correlation_threshold",
    "blocklist",
    "drop_na_columns",
]

# Columns to remove prior to single-cell aggregation via cameron's method
cameron_unwanted_aggregate_cols = {"Object", "Parent", "Site", "Image", "Location"}

# Set paths
output_dir = pathlib.Path("data/single_cell_profiles")
output_dir.mkdir(exist_ok=True)
metadata_dir = pathlib.Path("../0.download_data/metadata/")

# load in plate information
dictionary_path = pathlib.Path("./plate_info_dictionary.yaml")
with open(dictionary_path) as file:
    plate_info_dictionary = yaml.load(file, Loader=yaml.FullLoader)

In [3]:
# add path to platemaps for each plate
for plate in plate_info_dictionary.keys():
    # since Plate_3_prime has the same platemap as Plate_3,
    # we need an else statement so that we make sure it adds the
    # path that was given to Plate_3
    if plate != "Plate_3_prime":
        # match the naming format of the plates to the platemap file
        plate_info_dictionary[plate]["platemap_path"] = str(
            pathlib.Path(
                list(
                    metadata_dir.rglob(
                        f"platemap_NF1_{plate.replace('_', '').lower()}.csv"
                    )
                )[0]
            ).resolve(strict=True)
        )
    else:
        plate_info_dictionary["Plate_3_prime"]["platemap_path"] = plate_info_dictionary[
            "Plate_3"
        ]["platemap_path"]

# view the dictionary to assess that all info is added correctly
pprint.pprint(plate_info_dictionary, indent=4)

{   'Plate_3': {   'cleaned_path': 'data/cleaned_profiles/Plate_3_cleaned.parquet',
                   'dest_path': 'data/converted_data/Plate_3.parquet',
                   'platemap_path': '/media/18tbdrive/1.Github_Repositories/nf1_schwann_cell_painting_data/0.download_data/metadata/platemap_NF1_plate3.csv',
                   'source_path': '/media/18tbdrive/1.Github_Repositories/nf1_schwann_cell_painting_data/2.cellprofiler_analysis/analysis_output/Plate_3/Plate_3_nf1_analysis.sqlite'},
    'Plate_3_prime': {   'cleaned_path': 'data/cleaned_profiles/Plate_3_prime_cleaned.parquet',
                         'dest_path': 'data/converted_data/Plate_3_prime.parquet',
                         'platemap_path': '/media/18tbdrive/1.Github_Repositories/nf1_schwann_cell_painting_data/0.download_data/metadata/platemap_NF1_plate3.csv',
                         'source_path': '/media/18tbdrive/1.Github_Repositories/nf1_schwann_cell_painting_data/2.cellprofiler_analysis/analysis_output/Plate_3_p

## Perform single-cell pycytominer pipeline

In [4]:
# Ensure output_dir is set correctly before the loop
if data_level == "cleaned":
    output_dir = pathlib.Path(output_dir) / "cleaned_sc_profiles"
    output_dir.mkdir(parents=True, exist_ok=True)

for plate, info in plate_info_dictionary.items():
    print(f"Now performing single-cell pycytominer pipeline for {plate}")
    output_annotated_file = str(
        pathlib.Path(f"{output_dir}/{plate}_sc_annotated.parquet")
    )
    output_normalized_file = str(
        pathlib.Path(f"{output_dir}/{plate}_sc_normalized.parquet")
    )
    output_feature_select_file = str(
        pathlib.Path(f"{output_dir}/{plate}_sc_feature_selected.parquet")
    )
    output_aggregated_file = str(
        pathlib.Path(f"{output_dir}/{plate}_bulk_camerons_method.parquet")
    )

    # Load single-cell profiles
    if data_level == "cleaned":
        single_cell_df = pd.read_parquet(info["cleaned_path"])
    elif data_level == "converted":
        single_cell_df = pd.read_parquet(info["dest_path"])

    # Load plate map
    platemap_df = pd.read_csv(info["platemap_path"])

    # Step 1: Annotation
    # add metadata from plate map file to extracted single cell features
    annotated_df = annotate(
        profiles=single_cell_df,
        platemap=platemap_df,
        join_on=["Metadata_well_position", "Image_Metadata_Well"],
    )

    # rename site column to avoid any issues with identifying the column as metadata over feature
    annotated_df = annotated_df.rename(columns={"Image_Metadata_Site": "Metadata_Site"})

    # move metadata well, single cell count, and site to the front of the df (for easy visualization in python)
    well_column = annotated_df.pop("Metadata_Well")
    singlecell_column = annotated_df.pop("Metadata_number_of_singlecells")
    site_column = annotated_df.pop("Metadata_Site")

    # insert the columns in specific parts of the data frame
    annotated_df.insert(2, "Metadata_Well", well_column)
    annotated_df.insert(3, "Metadata_Site", site_column)
    annotated_df.insert(4, "Metadata_number_of_singlecells", singlecell_column)

    # for plates 5, 3, and 3 prime, remove any rows with HET due to contamination/not using in model
    if plate in ["Plate_5", "Plate_3", "Plate_3_prime"]:
        # Filter single-cell profiles, removing HET genotype
        annotated_df = annotated_df[annotated_df["Metadata_genotype"] != "HET"].reset_index(drop=True)
        print("HET cells have been removed from", plate)

    # use output to save annotated df as you can not use the output parameter in the annotate which will not return data frame
    output(
        df=annotated_df,
        output_filename=output_annotated_file,
        output_type="parquet",
    )
    print("Annotated dataframe shape", annotated_df.shape)

    # set default for samples to use in normalization and feature selection
    samples = "all"

    # Only for Plate 4, we want to normalize to no siRNA treatment Null and WT cells (controls)
    if plate == "Plate_4":
        samples = "Metadata_Concentration == 0.0 and (Metadata_genotype == 'Null' or Metadata_genotype == 'WT')"

    # Only for Plate 6, we want to normalize to iNFixion institution and Null and WT cells 
    # to keep consistent with how the other plates are normalized (same cell line)
    if plate == "Plate_6":
        samples = "Metadata_Institution == 'iNFixion' and (Metadata_genotype == 'Null' or Metadata_genotype == 'WT')"

    print(f"Performing normalization for {plate} using samples parameter: {samples}")

    # Step 2: Normalization
    normalized_df = normalize(
        profiles=output_annotated_file,
        method="standardize",
        samples=samples,
    )
    print("Normalized dataframe shape", normalized_df.shape)

    output(
        df=normalized_df,
        output_filename=output_normalized_file,
        output_type="parquet",
    )

    # Step 3: Feature selection
    feature_select_df = feature_select(
        output_normalized_file,
        operation=feature_select_ops,
        na_cutoff=0,
        samples=samples,
    )

    print("Feature selected dataframe shape", feature_select_df.shape)

    output(
        df=feature_select_df,
        output_filename=output_feature_select_file,
        output_type="parquet",
    )

    # Step 4: Cameron's method of aggregation
    # Specify metadata columns in aggregation step to ensure they are retained for downstream analysis
    metadata_cols = infer_cp_features(feature_select_df, metadata=True)
    metadata_cols = [
        x
        for x in metadata_cols
        if all(col not in x for col in cameron_unwanted_aggregate_cols)
    ]

    aggregate_df = aggregate(
        population_df=feature_select_df,
        operation="median",
        strata=metadata_cols,
    )

    output(
        df=aggregate_df,
        output_filename=output_aggregated_file,
        output_type="parquet",
    )

    print("Aggregated dataframe shape", aggregate_df.shape)

Now performing single-cell pycytominer pipeline for Plate_3
HET cells have been removed from Plate_3
Annotated dataframe shape (10206, 2326)
Performing normalization for Plate_3 using samples parameter: all
Normalized dataframe shape (10206, 2318)
Feature selected dataframe shape (10206, 1155)
Aggregated dataframe shape (48, 1145)
Now performing single-cell pycytominer pipeline for Plate_3_prime
HET cells have been removed from Plate_3_prime
Annotated dataframe shape (5126, 2326)
Performing normalization for Plate_3_prime using samples parameter: all
Normalized dataframe shape (5126, 2318)
Feature selected dataframe shape (5126, 1143)
Aggregated dataframe shape (48, 1133)
Now performing single-cell pycytominer pipeline for Plate_5
HET cells have been removed from Plate_5
Annotated dataframe shape (5348, 2325)
Performing normalization for Plate_5 using samples parameter: all
Normalized dataframe shape (5348, 2317)
Feature selected dataframe shape (5348, 1159)
Aggregated dataframe shape 